In [1]:
import pandas as pd
import ebcdic
import codecs

file = r'C:\PublicData\Texas\TXRRC\index\dbf900.ebc' ##Local storage location
##file origin: ftp://ftpe.rrc.texas.gov/shfwba/dbf900.ebc.gz

with open(file, 'rb') as ebcdic:
    data = ebcdic.read()

ascii_txt = codecs.decode(data, 'cp1140')

In [2]:
split_records = []
n  = 247 ##Unknown if this holds true for all versions of this file or for other files on TXRRC

for index in range(0, len(ascii_txt), n):
    split_records.append(ascii_txt[index : index + n])

#### Checking different fields and the formats
The lat and long data is a COBOL comp3 format decimal PIC S9(3)V9(7) which should translate to DDD.DDDDDDD

Getting values in the last position that are letters or "{" that do not make sense. From;
http://3480-3590-data-conversion.com/article-packed-fields.html
the last position should indicate the sign of the number, but there isn't a consistent decoding to make that work out.

The TXRRC map shows a 7decimal place percision when exported, but only 6 when reviewed though the info tool, these might be coming from different file sources. The 6decimal percision can be grabbed from the current field without the last letter code

In [21]:
API = None
ct = 0

sample_records = split_records[4000000:5200000]

for record in sample_records:
    if record.startswith('01'):
        API = '42'+record[2:10]
    if record.startswith('13'):
        #print(record)
        #print(ct,'API:',API,'lat:',record[132:142],'long:',record[142:152])
        try:
            #Checking to see if value for WB-PLANE-COORDINATE-EAST can be converted to number
            #This is for testing to check the coordinate reference system in file
            val = int(record[159:168])
        except:
            val = 0
            
        if val > 0:
            
            print(ct,'API:',API,'check:',record[152:])
            
            #WGS84 lat-long
            lat = float(str(int(record[132:135]))+'.'+str(int(record[135:141])))
            long = float('-'+str(int(record[142:145]))+'.'+str(int(record[145:151])))
            
            ##Unknown what coordinate reference system these are in
            coordZone = record[157:159]
            coordE = float(str(int(record[159:167]))+'.'+str(int(record[167])))
            coordN = float(str(int(record[169:177]))+'.'+str(int(record[177])))
            
            print(ct, API, "(" + str(lat) + ", "+ str(long) +")", "("+str(coordZone)+" "+str(coordE) + ", "+ str(coordN)+")")
            
        ct+=1

1063 API: 4210334286 check:      03013668190{007254970{Y0                                                                  
1063 4210334286 (31.645677, -102.36856) (03 1366819.0, 725497.0)
1089 API: 4210334312 check:      03013692980{007234250{Y0                                                                  
1089 4210334312 (31.639793, -102.363698) (03 1369298.0, 723425.0)
1100 API: 4210334323 check:      03013799500{007150400{Y0                                                                  
1100 4210334323 (31.61775, -102.325234) (03 1379950.0, 715040.0)
1101 API: 4210334324 check:      03013799750{007157700{Y0                                                                  
1101 4210334324 (31.622356, -102.325796) (03 1379975.0, 715770.0)
1102 API: 4210334325 check:      03013811900{007173630{Y0                                                                  
1102 4210334325 (31.624079, -102.321936) (03 1381190.0, 717363.0)
1103 API: 4210334326 check:      03013802000{0071412

12970 4210537464 (30.509475, -101.525122) (03 1624828.5, 308462.8)
12971 API: 4210537465 check:      03016959724{003271814{Y0                                                                  
12971 4210537465 (30.562832, -101.300675) (03 1695972.4, 327181.4)
12976 API: 4210537470 check:      03016527670{002651455{Y0                                                                  
12976 4210537470 (30.391168, -101.435043) (03 1652767.0, 265145.5)
12977 API: 4210537471 check:      03016570857{002654611{Y0                                                                  
12977 4210537471 (30.392152, -101.421355) (03 1657085.7, 265461.1)
12978 API: 4210537472 check:      03015670040{005112300{Y0                                                                  
12978 4210537472 (31.65157, -101.716789) (03 1567004.0, 511230.0)
12979 API: 4210537473 check:      03016835196{002893836{Y0                                                                  
12979 4210537473 (30.458611, -101.338202)

32673 API: 4212331812 check:      04026230440{004854550{Y0                                                                  
32673 4212331812 (29.154456, -97.48093) (04 2623044.0, 485455.0)
32692 API: 4212331831 check:      04026005840{004404630{Y0                                                                  
32692 4212331831 (29.31742, -97.120722) (04 2600584.0, 440463.0)
36308 API: 4212530591 check:      02009237047{006554930{Y0                                                                  
36308 4212530591 (33.418853, -101.28502) (02 923704.7, 655493.0)
36309 API: 4212530592 check:      02010406570{006795100{Y0                                                                  
36309 4212530592 (33.495037, -100.647721) (02 1040657.0, 679510.0)
39497 API: 4212733070 check:      04017691286E002673575FY0                                                                  
39497 4212733070 (28.566948, -99.719567) (04 1769128.6, 267357.5)


In [18]:
len(split_records)

25198781